In [1]:
import jax
import inspeqtor.experimental as sq

In [2]:
key = jax.random.key(0)
key, data_key, model_key, train_key, gate_optim_key = jax.random.split(key, 5)
sample_size = 1000

qubit_info = sq.predefined.get_mock_qubit_information()


# NOTE pick the pulse sequence you want to use
# def get_pulse_sequence_fn():
#         return sq.predefined.get_drag_pulse_sequence(qubit_info)

get_pulse_sequence_fn = sq.predefined.get_multi_drag_pulse_sequence_v3

# NOTE: Simulate the experiment with some detuning noise
(
    exp_data,
    pulse_sequence,
    noisy_unitaries,
    signal_params_list,
    noisy_simulator,
    whitebox,
) = sq.predefined.generate_mock_experiment_data(
    key=data_key,
    sample_size=sample_size,
    shots=3000,
    strategy=sq.predefined.SimulationStrategy.SHOT,
    detune=0.001,
    get_pulse_sequence_fn=get_pulse_sequence_fn,
    get_qubit_information_fn=sq.predefined.get_mock_qubit_information,
)

# Prepare the data for training
loaded_data = sq.utils.prepare_data(
    exp_data=exp_data, pulse_sequence=pulse_sequence, whitebox=whitebox
)

# Choose the loss metric
metric = sq.model.LossMetric.WAEE
# Define trainanle function for hyperparameter tuning
trainable = sq.optimize.default_trainable_v3(
    pulse_sequence=loaded_data.pulse_sequence,
    metric=metric,
    experiment_identifier="test",
    hamiltonian=sq.predefined.rotating_transmon_hamiltonian,
)

# Hyperparameter tuning
results = sq.optimize.hypertuner(
    trainable=trainable,
    pulse_parameters=loaded_data.pulse_parameters,
    unitaries=loaded_data.unitaries,
    expectation_values=loaded_data.expectation_values,
    train_key=train_key,
    num_samples=10,  # NOTE: The number of samples to train
    search_algo=sq.optimize.SearchAlgo.OPTUNA,
    metric=metric,
)

# Get the best hyperparameters
model_state, train_hist, data_config = sq.optimize.get_best_hypertuner_results(
    results, metric=metric
)

(trainable pid=87271) /Users/porametpathumsoot/University/PhD/Projects/espresso/inspeqtor/src/inspeqtor/experimental/model.py:46: UserWarning: Explicitly requested dtype <class 'numpy.complexfloating'> requested in zeros is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
(trainable pid=87271)   Q = jnp.zeros(U.shape[:-1] + (2, 2), dtype=jnp.complexfloating)
(trainable pid=87271) /Users/porametpathumsoot/University/PhD/Projects/espresso/inspeqtor/src/inspeqtor/experimental/model.py:57: UserWarning: Explicitly requested dtype <class 'numpy.complexfloating'> requested in zeros is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for mo

In [ ]:
# Save model and load model
save_path = sq.model.save_model(
    path="ckpt",
    experiment_identifier="test",
    pulse_sequence=loaded_data.pulse_sequence,
    hamiltonian=data_config.hamiltonian,
    model_config=model_state.model_config,
    model_params=model_state.model_params,
    history=train_hist,
)

loaded_model = sq.model.load_model(save_path)